In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\SAIDT.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using SAIDT;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

In [2]:
string dbPath = @"SAIDT_db" ;
var database = OpenOrCreateDatabase(dbPath);

Opening existing database 'SAIDT_db'.


In [3]:
Dictionary<string,int> DOFS = new Dictionary<string,int>();


## SinglePhaseField

In [4]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 100,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SinglePhaseField,
                    LSDegree: 3
                    );
CSAIDT.SessionName="SAIDT_DGLevelSet";
CSAIDT.ProjectName="SAIDT_DGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SinglePhaseField",pSAIDT.LsTBO.DOFLocal);

rm SAIDT_CGLevelSet_SP_0.plt SAIDT_CGLevelSet_SP_1.plt SAIDT_CGLevelSet_SP_10.plt SAIDT_CGLevelSet_SP_11.plt SAIDT_CGLevelSet_SP_12.plt SAIDT_CGLevelSet_SP_13.plt SAIDT_CGLevelSet_SP_14.plt SAIDT_CGLevelSet_SP_15.plt SAIDT_CGLevelSet_SP_16.plt SAIDT_CGLevelSet_SP_17.plt SAIDT_CGLevelSet_SP_18.plt SAIDT_CGLevelSet_SP_19.plt SAIDT_CGLevelSet_SP_2.plt SAIDT_CGLevelSet_SP_20.plt SAIDT_CGLevelSet_SP_21.plt SAIDT_CGLevelSet_SP_22.plt SAIDT_CGLevelSet_SP_23.plt SAIDT_CGLevelSet_SP_24.plt SAIDT_CGLevelSet_SP_25.plt SAIDT_CGLevelSet_SP_26.plt SAIDT_CGLevelSet_SP_27.plt SAIDT_CGLevelSet_SP_28.plt SAIDT_CGLevelSet_SP_29.plt SAIDT_CGLevelSet_SP_3.plt SAIDT_CGLevelSet_SP_30.plt SAIDT_CGLevelSet_SP_31.plt SAIDT_CGLevelSet_SP_32.plt SAIDT_CGLevelSet_SP_33.plt SAIDT_CGLevelSet_SP_34.plt SAIDT_CGLevelSet_SP_35.plt SAIDT_CGLevelSet_SP_36.plt SAIDT_CGLevelSet_SP_37.plt SAIDT_CGLevelSet_SP_38.plt SAIDT_CGLevelSet_SP_39.plt SAIDT_CGLevelSet_SP_4.plt SAIDT_CGLevelSet_SP_40.plt SAIDT_CGLevelSet_SP_41.plt SAI

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         75.13101109216082
    L2 Norm of Solution:    0.03375347277990737
    L2 Norm of Residual:    0.0008282958526022142
    Relative Residual norm: 2.6467892807400498E-09
    Matrix Inf norm:        312943.6327362715



alpha, M, predM  1, 2.6715087502151635, 0.2577553924823621
alpha, M, predM  0.5, 0.6786828472960703, 0.2577800728986183
alpha, M, predM  0.25, 1.4815890757345038, 0.25779241310674644
alpha, M, predM  0.125, 1.0427797466593882, 0.2577985832108105
alpha, M, predM  0.0625, 1.4107041749411782, 0.25780166826284256
alpha, M, predM  0.03125, 2.2051646108828864, 0.25780321078885854
alpha, M, predM  0.015625, 3.575048704799327, 0.25780398205186655
alpha, M, predM  0.0078125, 5.387608998913839, 0.25780436768337056
alpha, M, predM  0.00390625, 7.369624022934664, 0.2578045604991226
alpha, M, predM  0.001953125, 9.111843681636337, 0.25780465690699855
alpha, M, predM  0.0009765625, 10.365654860391567, 0.25780470511093656
alpha, M, predM  0.00048828125, 11.14211121865762, 0.2578047292129056
alpha, M, predM  0.000244140625, 11.578722085377692, 0.2578047412638901
alpha, M, predM  0.0001220703125, 11.810942229635542, 0.2578047472893823
alpha, M, predM  6.103515625E-05, 11.930795567401628, 0.257804750302

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         75.13099328435521
    L2 Norm of Solution:    0.03504686200603139
    L2 Norm of Residual:    0.0010198124282419041
    Relative Residual norm: 3.2587741840170673E-09
    Matrix Inf norm:        312943.57039025903



alpha, M, predM  1, 2.667280419075224, 0.2577938910585642
alpha, M, predM  0.5, 0.7002181048579955, 0.257818570809348
alpha, M, predM  0.25, 1.461443562491215, 0.2578309106847399
alpha, M, predM  0.125, 1.038881542346215, 0.2578370806224358
alpha, M, predM  0.0625, 1.406195468430461, 0.2578401655912838
alpha, M, predM  0.03125, 2.1934388767615896, 0.25784170807570783
alpha, M, predM  0.015625, 3.5609204732689057, 0.2578424793179198
alpha, M, predM  0.0078125, 5.372040973389388, 0.2578428649390258
alpha, M, predM  0.00390625, 7.354934372736156, 0.2578430577495788
alpha, M, predM  0.001953125, 9.100140354476304, 0.2578431541548553
alpha, M, predM  0.0009765625, 10.357414977690365, 0.25784320235749353
alpha, M, predM  0.00048828125, 11.136575329547753, 0.2578432264588127
alpha, M, predM  0.000244140625, 11.574895944804291, 0.2578432385094722
alpha, M, predM  0.0001220703125, 11.808081169465932, 0.257843244534802
alpha, M, predM  6.103515625E-05, 11.928447771863635, 0.2578432475474669
alph

...

alpha, M, predM  1, 9.992574481429424, 0.19455407496026855
alpha, M, predM  0.5, 7.681943226617647, 0.19457349756852793
alpha, M, predM  0.25, 6.265490633110455, 0.19458320887265762
alpha, M, predM  0.125, 4.105040293111354, 0.19458806452472246
alpha, M, predM  0.0625, 1.4087756200916146, 0.19459049235075487
alpha, M, predM  0.03125, 1.417668284351604, 0.1945917062637711
alpha, M, predM  0.015625, 1.2250015208721852, 0.1945923132202792
alpha, M, predM  0.0078125, 1.4025716425131134, 0.19459261669853326
alpha, M, predM  0.00390625, 1.5117400011348365, 0.1945927684376603
alpha, M, predM  0.001953125, 1.5778719985115095, 0.1945928443072238
alpha, M, predM  0.0009765625, 1.6298749052138086, 0.19459288224200555
alpha, M, predM  0.00048828125, 1.6560757898054497, 0.19459290120939643
alpha, M, predM  0.000244140625, 1.6692222797884273, 0.19459291069309187
alpha, M, predM  0.0001220703125, 1.6758065514541869, 0.1945929154349396
alpha, M, predM  6.103515625E-05, 1.6791013827636534, 0.1945929178

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         177083748.55143556
    L2 Norm of Solution:    25.508553592324674
    L2 Norm of Residual:    42464725.56148154
    Relative Residual norm: 2.2888080605939227E-10
    Matrix Inf norm:        1.8553205178097098E+17



alpha, M, predM  1, 13.127578943799906, 0.20946015196018433
alpha, M, predM  0.5, 9.657090082092264, 0.20947881076464703
alpha, M, predM  0.25, 5.220325986291452, 0.2094881401668784
alpha, M, predM  0.125, 2.0924361540602945, 0.20949280486799407
alpha, M, predM  0.0625, 1.6897416943529184, 0.20949513721855192
alpha, M, predM  0.03125, 1.7113715361144937, 0.20949630339383082
alpha, M, predM  0.015625, 1.6836862798969272, 0.2094968864814703
alpha, M, predM  0.0078125, 1.6936232757275667, 0.20949717802529003
alpha, M, predM  0.00390625, 1.6993990808936619, 0.20949732379719987
alpha, M, predM  0.001953125, 1.7031233561344996, 0.2094973966831548
alpha, M, predM  0.0009765625, 1.7050768747085843, 0.20949743312613228
alpha, M, predM  0.00048828125, 1.7060572710178787, 0.20949745134762102
alpha, M, predM  0.000244140625, 1.7065483827229395, 0.2094974604583654
alpha, M, predM  0.0001220703125, 1.7067941675123353, 0.20949746501373756
alpha, M, predM  6.103515625E-05, 1.7069171172107285, 0.209497

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         177083734.00578845
    L2 Norm of Solution:    412.1884145617006
    L2 Norm of Residual:    183466897.52331793
    Relative Residual norm: 9.888691921533075E-10
    Matrix Inf norm:        1.855320187736969E+17



alpha, M, predM  1, 6.758303077084886, 0.19535157389817673
alpha, M, predM  0.5, 4.473517706877492, 0.19537097613344526
alpha, M, predM  0.25, 1.7262292591207407, 0.19538067725107952
alpha, M, predM  0.125, 1.6933536108663472, 0.19538552780989665
alpha, M, predM  0.0625, 0.23587175912069785, 0.19538795308930523
alpha, M, predM  0.03125, 0.19441544148045434, 0.1953891657290095
l2: ||R0||= 2.6068646E-01, f_err=6.2209E-01, f_phi=0E+00, Alpha = 3.13E-02, Gamma = 1E-04, Kappa = 1E+03, Mu = 1.56E-03, QuadOrder=4

ALNR=1 <= 1.005 so we terminate/increase degree

ALNR=1.0003176680792876 <= 1.005 so we terminate/increase degree



In [5]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)||

In [6]:
var si= database.Sessions.Pick(0);
BoSSS.Solution.Application.DeleteOldPlotFiles();
pSAIDT.PlotShadowFields(si);


rm;


## Spline level set

In [7]:
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 100,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
                    LSDegree: 3,
                    isFarConfig:true
                    );
CSAIDT.SessionName="SAIDT_SplineLevelSet";
CSAIDT.ProjectName="SAIDT_SplineLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SplineLevelSet",pSAIDT.LevelSetOpti.GetLength());


Session ID: 12384742-a97b-432a-ba36-1b0378e41970, DB path: 'SAIDT_db'
Session directory 'SAIDT_db\sessions\12384742-a97b-432a-ba36-1b0378e41970'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_SplineLevelSet
*  session:                            SAIDT_SplineLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:               

In [8]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 10 
 
 
 
 
 20 
 
 
 
 
 30 
 
 
 
 
 40 
 
 
 
 
 50 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M388.5,57.1 L441.9,57.1 M53.9,88.9 L62.2,91.6 L70.4,93.4 L78.7,94.6 L87.0,94.7 L95.3,96.5
 L103.5,96.8 L111.8,96.8 L120.1,96.9 L128.3,96.9 L136.6,96.9 L144.9,96.9 L153.1,96.9 L161.4,96.9
 L169.7,96.9 L178.0,96.9 L186.2,104.8 L194.5,105.8 L202.8,112.7 L211.0,126.1 L219.3,129.2 L227.6,129.7
 L235.8,129.8 L244.1,129.8 L252.4,129.8 L260.7,129.8 L268.9,153.2 L277.2,188.8 L285.5,215.9 L293.7,251.2
 L302.0,287.1 L310.3,327.7 L318.5,345.8 L326.8,352.5 L335.1,356.6 L343.4,365.4 L351.6,369.6 L359.9,371.4
 L368.2,371.3 L376.4,371.0 L384.7,372.5 L393.0,372.4 L401.2,373.7 L409.5,373.3 L417.8,374.5 L426.1,374.3
 L434.3,376.9 L442.6,375.2 L450.9,378.1 L459.1,376.8 L467.4,377.2 L475.7,377.4 L483.9,377.4 L492.2,377.4
 L500.5,377.5 L508.8,377.5 L517.0,377.5 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M388.5,81.1 L441.9,81.1 M53.9,113.1 L62.2,118.7 L70.4,123.3 L78.7,115.3 L87.0,121.7 L95.3,125.4
 L103.5,126.2 L111.8,126.4 L120.1,126.5 L128.3,126.5 L136.6,126.5 L144.9,126.5 L153.1,126.5 L161.4,126.5
 L169.7,126.5 L178.0,126.5 L186.2,114.4 L194.5,114.9 L202.8,121.8 L211.0,142.6 L219.3,147.2 L227.6,147.9
 L235.8,148.0 L244.1,148.0 L252.4,148.0 L260.7,148.1 L268.9,169.1 L277.2,207.0 L285.5,232.9 L293.7,269.7
 L302.0,304.3 L310.3,340.6 L318.5,351.0 L326.8,358.4 L335.1,362.4 L343.4,379.0 L351.6,381.8 L359.9,393.6
 L368.2,393.5 L376.4,391.7 L384.7,396.8 L393.0,393.2 L401.2,397.7 L409.5,397.0 L417.8,396.6 L426.1,400.3
 L434.3,394.6 L442.6,403.3 L450.9,395.3 L459.1,406.5 L467.4,404.3 L475.7,402.9 L483.9,402.6 L492.2,402.4
 L500.5,402.4 L508.8,402.4 L517.0,402.4 '/>

In [9]:
var si= database.Sessions.Pick(0);
pSAIDT.PlotShadowFields(si);

## SpecFemField

In [19]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 100,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: 5,
                    optiLevelSetType: OptiLevelSetType.SpecFemField,
                    LSDegree: 3,
                    isFarConfig:true
                    );
                    
CSAIDT.LevelSetDegree = CSAIDT.OptiLevelSetDegree*2;
CSAIDT.SessionName="SAIDT_CGLevelSet";
CSAIDT.ProjectName="SAIDT_CGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SpecFemField",pSAIDT.LevelSetOpti.GetLength());

Session ID: d322f909-4e60-46e8-b68c-2395d1a22e53, DB path: 'SAIDT_db'
Session directory 'SAIDT_db\sessions\d322f909-4e60-46e8-b68c-2395d1a22e53'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_CGLevelSet
*  session:                            SAIDT_CGLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:                     Sp

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         8.547379345699525
    L2 Norm of Solution:    1.3340356587376387
    L2 Norm of Residual:    1.679725129127782E-06
    Relative Residual norm: 1.4704505849517812E-09
    Matrix Inf norm:        1142.3200115105285



alpha, M, predM  1, 0.6836421982596652, 0.9199499237170777
l2: ||R0||= 9.3269297E-02, f_err=9.258416E-01, f_phi=0E+00, Alpha = 1E+00, Gamma = 1E+00, Kappa = 1E+03, Mu = 1.31E+00, QuadOrder=7
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.11: 
alpha, M, predM  1, 1.935674136146988, 0.6156547894661868
alpha, M, predM  0.5, 1.0563450449704972, 0.6156609964254739
alpha, M, predM  0.25, 0.7317813756539886, 0.6156640999051174
alpha, M, predM  0.125, 0.5836722384949405, 0.6156656516449392
l2: ||R0||= 8.8550176E-02, f_err=8.771549E-01, f_phi=0E+00, Alpha = 1.25E-01, Gamma = 1E+00, Kappa = 1E+03, Mu = 1E+00, QuadOrder=7
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.12: 
alpha, M, predM  1, 0.8770685056703995, 0.6955796181649755
alpha, M, predM  0.5, 0.6703892244979313, 0.6955743893852692
l2: ||R0||= 1.0810908E-01, f_err=8.352305E-01, f_phi=0E+00, Alpha = 5E-01, Gamma = 1E+00, Kappa = 1E+03, Mu = 1.47E+00, QuadO

Error: System.ArgumentException: An item with the same key has already been added. Key: SpecFemField
   at System.Collections.Generic.Dictionary`2.TryInsert(TKey key, TValue value, InsertionBehavior behavior)
   at System.Collections.Generic.Dictionary`2.Add(TKey key, TValue value)
   at Submission#20.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [20]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -16 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 10 
 
 
 
 
 20 
 
 
 
 
 30 
 
 
 
 
 40 
 
 
 
 
 50 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M388.5,57.1 L441.9,57.1 M53.9,77.9 L62.2,78.8 L70.4,80.4 L78.7,79.2 L87.0,79.4 L95.3,80.2
 L103.5,79.2 L111.8,80.1 L120.1,80.3 L128.3,80.6 L136.6,82.0 L144.9,82.5 L153.1,83.0 L161.4,83.4
 L169.7,83.9 L178.0,84.4 L186.2,82.9 L194.5,83.8 L202.8,84.4 L211.0,84.9 L219.3,85.6 L227.6,85.6
 L235.8,84.2 L244.1,85.6 L252.4,85.6 L260.7,85.5 L268.9,86.1 L277.2,86.5 L285.5,87.0 L293.7,87.3
 L302.0,87.4 L310.3,88.2 L318.5,88.7 L326.8,88.6 L335.1,89.0 L343.4,88.9 L351.6,88.3 L359.9,88.9
 L368.2,89.5 L376.4,89.6 L384.7,90.6 L393.0,90.6 L401.2,90.2 L409.5,90.9 L417.8,91.0 L426.1,91.6
 L434.3,91.6 L442.6,91.9 L450.9,91.9 L459.1,91.9 L467.4,92.0 L475.7,92.1 L483.9,92.1 L492.2,92.2
 L500.5,92.2 L508.8,92.3 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M388.5,81.1 L441.9,81.1 M53.9,424.2 L62.2,105.0 L70.4,108.4 L78.7,103.5 L87.0,103.8 L95.3,103.4
 L103.5,101.4 L111.8,101.3 L120.1,102.4 L128.3,101.6 L136.6,104.5 L144.9,105.0 L153.1,103.1 L161.4,103.9
 L169.7,102.5 L178.0,100.7 L186.2,99.7 L194.5,99.7 L202.8,98.0 L211.0,98.9 L219.3,100.0 L227.6,100.0
 L235.8,99.5 L244.1,96.4 L252.4,97.9 L260.7,100.1 L268.9,100.0 L277.2,99.8 L285.5,100.0 L293.7,99.8
 L302.0,99.4 L310.3,100.5 L318.5,101.6 L326.8,100.8 L335.1,101.4 L343.4,99.8 L351.6,101.1 L359.9,102.4
 L368.2,103.7 L376.4,101.2 L384.7,101.7 L393.0,101.7 L401.2,108.0 L409.5,104.9 L417.8,105.2 L426.1,103.8
 L434.3,107.5 L442.6,106.7 L450.9,108.4 L459.1,108.2 L467.4,108.3 L475.7,109.6 L483.9,109.9 L492.2,110.5
 L500.5,111.0 L508.8,111.0 '/>

In [28]:
var si= database.Sessions.Pick(0);
pSAIDT.PlotShadowFields(si);

In [22]:
using System.IO;
static void AppendRowToCSV(string filePath, double[] rowData)
    {
        // Create a StreamWriter in append mode to append rows to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath, true))
        {
            int cols = rowData.Length;

            // Write double values of the new row to the file
            for (int i = 0; i < cols; i++)
            {
                writer.Write(rowData[i]);

                // Add a comma after each value except for the last one in a row
                if (i < cols - 1)
                {
                    writer.Write(";");
                }
            }

            // Move to the next line after writing the row
            writer.WriteLine();
        }
    }
    static void WriteCSV(string filePath, string[] data)
    {
        // Create a StreamWriter to write to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath))
        {
            int cols = data.Length;

            // Loop through the data array and write to the file
                for (int j = 0; j < cols; j++)
                {
                    // Write each cell value to the file
                    writer.Write(data[ j]);

                    // Add a comma after each cell value except for the last one in a row
                    if (j < cols - 1)
                    {
                        writer.Write(";");
                    }
                }

                // Move to the next line after each row
                writer.WriteLine();
            
        }
    }
    static void DeleteCSVFiles()
    {
        try
        {
            // Get all CSV files in the specified directory
            string[] csvFiles = Directory.GetFiles("*.csv");

            // Delete each CSV file
            foreach (string file in csvFiles)
            {
                File.Delete(file);
                Console.WriteLine($"Deleted: {file}");
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error deleting CSV files: {ex.Message}");
        }
    }

## Measure wall-clock time for computation of sensitivities and residuals 

In [23]:
using System.Diagnostics;
BoSSS.Solution.Application.InitMPI();
DeleteCSVFiles();

int nRefs=4;
var optLsTypes=new OptiLevelSetType[] {OptiLevelSetType.SinglePhaseField,OptiLevelSetType.SplineLevelSet,OptiLevelSetType.SpecFemField};
var CellsVsRuntime_dRdPhi = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_Residuals = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_dRdU = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
string[] stringArray = new string[] { "nCells" }.Concat(optLsTypes.Select(x => x.ToString())).ToArray();

//Create files for results
string filePathdRdPhi = "TimeDRdPhi.csv";
string filePathdRdU = "TimedRdU.csv";
string filePathResiduals = "TimeResiduals.csv";
string filePathDOFs = "DOFs.csv";
WriteCSV(filePathdRdPhi,stringArray);
WriteCSV(filePathdRdU,stringArray);
WriteCSV(filePathResiduals,stringArray);
WriteCSV(filePathDOFs,stringArray);

for(int n =0; n< nRefs;n++){
    //counter for level set type
    int iLS=1;
    //save cell resolution
    int reso= (int) Math.Pow(2,n+1);
    CellsVsRuntime_dRdPhi[n,0]=reso*reso;
    double[] dRdPhiResults= new double[optLsTypes.Length +1];
    dRdPhiResults[0]=reso*reso;
    double[] dRdUResults=dRdPhiResults.CloneAs();
    double[] resiudalResults=dRdPhiResults.CloneAs();
    double[] DOFs=dRdPhiResults.CloneAs();
    foreach(OptiLevelSetType lstype in optLsTypes){
        var pSAIDT = new SAIDTMain();
        var CSAIDT = SAIDTHardCodedControl.StraightShock(
                            dbPath: null,
                            MaxIterations: 1,
                            dgDegree: 0,
                            numOfCellsX: reso,
                            numOfCellsY: reso,
                            OptiNumOfCellsX: reso,
                            OptiNumOfCellsY: reso,
                            agg: 0.4,
                            ImmediatePlotPeriod: -1,
                            optiLevelSetType: lstype,
                            LSDegree: 3,
                            isFarConfig:true);

        pSAIDT.Init(CSAIDT);    
        pSAIDT.InitializeEverything();
        //pSAIDT.RunSolverMode();

        //Measure time for creation of Jacobians dRdPhi
        Stopwatch stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.FD_LevelSet();
        stopwatch.Stop();
        CellsVsRuntime_dRdPhi[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdPhiResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for creation of Jacobians dRdU
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.Oproblem.GetJacobians(pSAIDT.ConservativeFields, pSAIDT.XSpatialOperator.LinearizationHint);
        stopwatch.Stop(); 
        CellsVsRuntime_dRdU[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdUResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for evaluation of the Residuals
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.ComputeResiduals();
        stopwatch.Stop(); Console.WriteLine("Elapsed Time: " + stopwatch.ElapsedMilliseconds + "ms");
        CellsVsRuntime_Residuals[n,iLS] =stopwatch.ElapsedMilliseconds;
        resiudalResults[iLS]=stopwatch.ElapsedMilliseconds;

        DOFs[iLS]=pSAIDT.LevelSetOpti.GetLength();

        iLS++;
    }
    // Append the new row to the existing CSV file
    AppendRowToCSV(filePathdRdPhi, dRdPhiResults);
    AppendRowToCSV(filePathdRdU, dRdUResults);
    AppendRowToCSV(filePathResiduals, resiudalResults);
    AppendRowToCSV(filePathDOFs, DOFs);
}

Error deleting CSV files: The filename, directory name, or volume label syntax is incorrect. : 'c:\experimental\public\examples\ShockFitting\Studies\ComparisonLevelSets\*.csv'
Session ID: 00000000-0000-0000-0000-000000000000, DB path: 'EMPTY'
Session directory 'EMPTY\sessions\00000000-0000-0000-0000-000000000000'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            ScalarAdvection
*  session:             

In [24]:
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

In [25]:
CellsVsRuntime_dRdPhi

Dimension,2
Lengths,"[ 4, 4 ]"
Storage,"[ 4, 1381, 254, 747, 16, 6010, 540, 3507, 64, 34052, 1149, 22244, 256, 282971, 4145, 184385 ]"
IsContinuous,True
StructureType,General
Length,16
NoOfCols,4
NoOfRows,4
IsLocked,False


## Plot time needed for computation of dRdPhi against number of cells

In [26]:
var plot = new Plot2Ddata();
int count=0;
for(int iLS=0;iLS<CellsVsRuntime_dRdPhi.Lengths[1]-1; iLS ++){
    plot.AddDataGroup(optLsTypes[iLS].ToString(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,0).To1DArray(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,iLS+1).To1DArray(), GetFormat(count));
    count++;
}
plot.Xlabel="number of Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 10 5 
 
 
 
 
 10 6 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 number of Cells 
 
 
 
 
 SinglePhaseField 
 
 
 SinglePhaseField 
 
 
 
 
 
 
 
 
 
 
 SplineLevelSet 
 
 
 SplineLevelSet 
 
 
 
 
 
 
 
 
 
 
 SpecFemField 
 
 
 SpecFemField